In [1]:
import pandas as pd
import seaborn as sns


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/rahulsharma-rks/DepressionDetection/main/depression_data.csv")
df.tail()

,Unnamed: 0,message,label
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1
10312,802312,dont mistake a bad day with depression! everyo...,1
10313,802313,0,1


In [3]:
df['message'].iloc[:1]


0    just had a real good moment. i missssssssss hi...
Name: message, dtype: object

In [4]:
df = df.drop('Unnamed: 0',axis=1)

In [5]:
import re
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
wo = WordNetLemmatizer()
corpus=[]
for i in range(0,len(df)):
    message = re.sub('[^a-zA-Z]',' ',df['message'][i])
    message = message.lower()
    message = message.split()
    message = [wo.lemmatize(word) for word in message ]
    message = ' '.join(message)
    corpus.append(message)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus,df['label'],test_size=0.25,random_state=42)


In [7]:
from transformers import BertTokenizer


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [10]:
#!pip install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
#!pip install torch
#!pip install http://download.pytorch.org/whl/cpu/torch-0.4.0-cp36-cp36m-win_amd64.whl 
#!pip install torchvision
#!pip install torch torchvision torchaudio
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [11]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')


In [12]:
result = model(tokens)


In [13]:
result.logits


tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [14]:
int(torch.argmax(result.logits))+1


4

In [15]:
def sentiment_score(message):
    tokens = tokenizer.encode(message, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [16]:
sentiment_score(df['message'].iloc[1])

3

In [20]:
sentiment_score("I am very sad")

1

In [16]:
df['sentiment'] = df['message'].apply(lambda x: sentiment_score(x[:512]))


In [30]:
df.head()

,message,label,sentiment
0,just had a real good moment. i missssssssss hi...,0,3
1,is reading manga http://plurk.com/p/mzp1e,0,3
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0,5
3,@lapcat Need to send 'em to my accountant tomo...,0,3
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,5


In [49]:
sentiment_score("I am fine   " )

5

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus,df['sentiment'],test_size=0.25,random_state=42)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer( ngram_range=(1,3), stop_words='english',max_features=15000)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [33]:
from imblearn import under_sampling 
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE
x_resample, y_resample = SMOTE().fit_resample(X_train_vect, y_train)
x_test_resample, y_test_resample = SMOTE().fit_resample(X_test_vect, y_test)
# lets print the shape of x and y after resampling it
print(x_resample.shape)
print(y_resample.shape)

(16610, 15000)
(16610,)


In [40]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_resample,y_resample)

MultinomialNB()

In [42]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = mnb.predict(x_test_resample)
accuracy_score(y_test_resample,y_pred)
print(classification_report(y_test_resample,y_pred))
#mnb.save("Multinomial.model", save_format="h5")
filename = 'Multinomial.pkl'
#pickle.dump(mnb, open(filename, 'wb'))

              precision    recall  f1-score   support

           1       0.40      0.39      0.40      1151
           2       0.44      0.57      0.49      1151
           3       0.38      0.22      0.28      1151
           4       0.45      0.25      0.32      1151
           5       0.40      0.64      0.49      1151

    accuracy                           0.41      5755
   macro avg       0.41      0.41      0.40      5755
weighted avg       0.41      0.41      0.40      5755

